### Utils

In [58]:
from time import sleep
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webelement import WebElement
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep

def is_bottom_of_page(driver: webdriver.Chrome) -> bool:
    # Get the current scroll position and the total scrollable height
    current_scroll_position = driver.execute_script("return window.pageYOffset + window.innerHeight;")
    total_scroll_height = driver.execute_script("return document.body.scrollHeight;")
    return current_scroll_position >= total_scroll_height

def infinite_scroll(driver: webdriver.Chrome):
    while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            sleep(2)
            
            check_showmore = False
            try:
                element = driver.find_element(By.CLASS_NAME, value='''artdeco-button artdeco-button--muted artdeco-button--1 artdeco-button--full artdeco-button--secondary ember-view scaffold-finite-scroll__load-button''')
                check_showmore = True

            except NoSuchElementException as e:
                check_showmore = False
                pass

            if check_showmore:
                driver.execute_script('window.scrollTo(document.body.scrollHeight, 0);')
                sleep(2)
                driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')

            if is_bottom_of_page(driver):
                break

def get_fullxpath(driver: webdriver.Chrome, element: WebElement) -> str:
    return driver.execute_script( 
     '''function getElementXPath(elt) {
            var path = "";
            for (; elt && elt.nodeType == 1; elt = elt.parentNode) {
                idx = getElementIdx(elt);
                xname = elt.tagName;
                if (idx > 1) xname += "[" + idx + "]";
                path = "/" + xname + path;
            }
            return path;
        }

        function getElementIdx(elt) {
            var count = 1;
            for (var sib = elt.previousSibling; sib; sib = sib.previousSibling) {
                if (sib.nodeType == 1 && sib.tagName == elt.tagName) count++;
            }
            return count;
        }

        return getElementXPath(arguments[0]);''',
        element
    ).lower()

def get_urls(driver):
    page_source = BeautifulSoup(driver.page_source)
    profiles = page_source.find_all('a', class_ = 'ember-view mn-connection-card__link')
    all_profile_URL = []
    for profile in profiles:
        profile_ID = profile.get('href')
        profile_URL = "https://www.linkedin.com" + profile_ID
        if profile_URL not in all_profile_URL:
            all_profile_URL.append(profile_URL)
    return all_profile_URL

### LinkedInCrawler class

In [39]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.remote.webelement import WebElement
from bs4 import BeautifulSoup
from time import sleep

# from utils import infinite_scroll, get_fullxpath, is_bottom_of_page, get_urls


In [61]:
class LinkedInCrawler:
    driver: webdriver.Chrome
    connection_urls: list
    connection_info_cache: list

    def __init__(self):
        self.driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
        self.connection_urls = []

    #region LOGIN LINKEDIN
    def login_linkedin(self, username, password):
        sleep(1.5)
        url = 'https://www.linkedin.com/checkpoint/lg/sign-in-another-account'
        self.driver.get(url)
        print('- Finish initializing a self.driver')
        sleep(1.5)

        password = "prodn123"
        username = "skytdv123@gmail.com"
        print('- Finish importing the login credentials')
        sleep(1.5)

        email_field = self.driver.find_element(by=By.ID, value="username")
        email_field.send_keys(username)
        print('- Finish keying in email')
        sleep(1.5)

        password_field = self.driver.find_element(by=By.NAME, value="session_password")
        password_field.send_keys(password)
        password_field.send_keys
        print('- Finish keying in password')
        sleep(1.5)

        signin_field = self.driver.find_element(by=By.XPATH, value='//*[@id="organic-div"]/form/div[3]/button')
        signin_field.click()
        print('- Finish Task: Login to Linkedin')
        sleep(2)
    #endregion


    #region GET CONNECTIONS URLs
    def get_connection_urls(self):
        sleep(1)
        # my_network_btn = self.driver.find_element(By.XPATH, value='//*[@id="global-nav"]/div/nav/ul/li[2]/a')
        # wait = WebDriverWait(self.driver, 10)
        # wait.until(EC.element_to_be_clickable(my_network_btn))
        # my_network_btn.click()

        # sleep(1)
        # connections_btn = self.driver.find_element(By.XPATH, value='//*[@id="root"]/div[3]/div[2]/div/div/div/section/div/div/section[1]/div/nav/ul/li[1]/a')
        # wait = WebDriverWait(self.driver, 10)
        # wait.until(EC.element_to_be_clickable(connections_btn))
        # connections_btn.click()
        self.driver.get('https://www.linkedin.com/mynetwork/invite-connect/connections/')
        #scroll loop
        while True:
                sleep(1)
                self.driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
                sleep(1)
                
                check_showmore = False
                try:
                    element = self.driver.find_element(By.CLASS_NAME, value='''artdeco-button artdeco-button--muted artdeco-button--1 artdeco-button--full artdeco-button--secondary ember-view scaffold-finite-scroll__load-button''')
                    check_showmore = True

                except NoSuchElementException as e:
                    check_showmore = False
                    pass

                if check_showmore:
                    self.driver.execute_script('window.scrollTo(document.body.scrollHeight, 0);')
                    self.driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')

                if is_bottom_of_page(self.driver):
                    print("Reached the bottom of the page.")
                    break


        urls = get_urls(self.driver)
        self.connection_urls = urls
    #endregion


    #region GET CONTACT INFO
    def get_contact_info(self):
        try:
            sleep(1)
            self.driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')     
            sleep(0.5)   
            self.driver.execute_script('window.scrollTo(document.body.scrollHeight,0);')
            sleep(0.5)
            a_contact_info = self.driver.find_element(By.ID, value='''top-card-text-details-contact-info''')
            wait = WebDriverWait(self.driver, 10)
            wait.until(EC.element_to_be_clickable(a_contact_info))
            a_contact_info.click()

            sleep(1.5)
            div_profile_section = self.driver.find_element(By.XPATH, value='''/html/body/div[4]/div/div''')
            div_profile_section_source = BeautifulSoup(div_profile_section.get_attribute('outerHTML'),'html.parser')
            h1_tags = div_profile_section_source.find_all('h1')
            h1_text = [tag.text.strip() for tag in h1_tags]
            h3_tags = div_profile_section_source.find_all('h3')
            h3_text = [tag.text.strip() for tag in h3_tags]
            a_tags = div_profile_section_source.find_all('a')
            a_text = [tag.text.strip() for tag in a_tags]
            span_tags = div_profile_section_source.find_all('span')
            span_text = [tag.text.strip() for tag in span_tags]
            
            exit_btn = self.driver.find_element(By.XPATH, value='''/html/body/div[4]/div/div/button''')
            exit_btn.click()

            return {
                'h1_tag_text': h1_text,
                'h3_tag_text': h3_text,
                'a_tag_text': a_text,
                'span_tang_text': span_text
            }
        except NoSuchElementException as e:
            print(e)    
    #endregion


    #region GET AVAILABLE SECTIONS ON A PROFILE
    def get_sections(self) -> dict:
        sleep(1)
        section_head = {}
        main_element = self.driver.find_element(By.CLASS_NAME, value='scaffold-layout__main')
        main_element_source = BeautifulSoup(main_element.get_attribute('outerHTML'),'html.parser')
        sections = main_element_source.find_all('section', class_='artdeco-card pv-profile-card break-words mt2')
        for i in range(0,len(sections)):
            if len([tag.text.strip() for tag in sections[i].select('span.visually-hidden')]) > 0:
                head = ([tag.text.strip() for tag in sections[i].select('span.visually-hidden')][0])
                section_head[head] = i+2
        return section_head
    #endregion


    #region GET SECTION's INFO
    def get_section_info(self, section_number: int, full_xpath: str ):
        sleep(2)
        wait = WebDriverWait(self.driver,10)
        info_list = []
        show_more_check = False
        try:
            show_more = self.driver.find_element(By.XPATH, value= full_xpath + f'''/section[{section_number}]/div[3]/div/div/div/a''')
            show_more_check = True
        except NoSuchElementException:
            show_more_check = False

        if show_more_check:   #if yes: click show more
            click_count = 0
            while(True):
                try:
                    self.driver.execute_script(f"window.scrollTo(0, {show_more.size['height']});")
                    wait.until(EC.element_to_be_clickable(show_more))
                    show_more.click()
                    sleep(2)
                except Exception as e:
                    click_count += 1
                    print(e)
                
                if click_count > 0:
                    continue
                else:
                    break

            infinite_scroll(self.driver)
            list_container = self.driver.find_element(By.CLASS_NAME, value='pvs-list__container')
            list_container_source = BeautifulSoup(list_container.get_attribute('outerHTML'),'html.parser')
            li_elements = list_container_source.find_all('div', class_='display-flex flex-column align-self-center flex-grow-1')

            for li in li_elements:
                span_tags = li.find_all('span', class_='visually-hidden')
                span_text = [tag.text.strip() for tag in span_tags]
                info_list.append(span_text) 
                    
            self.driver.back()
            
        else:
            fail_count = 0
            while (fail_count):
                try:
                    ul_element = self.driver.find_element(By.XPATH, value=f'//*[@id="profile-content"]/div/div[2]/div/div/main/section[{section_number}]/div[3]/ul')
                except NoSuchElementException as e:
                    fail_count += 1

            if(fail_count==5):
                ul_element = self.driver.find_element(By.XPATH, value=f'//*[@id="profile-content"]/div/div[2]/div/div/main/section[{section_number}]/div[3]/ul')
                

            ul_element_source = BeautifulSoup(ul_element.get_attribute('outerHTML'),'html.parser')
            number_of_li = len(ul_element_source.find_all('li',class_='artdeco-list__item'))

            if(number_of_li<2):
                # li_element = driver.find_element(By.XPATH, value='/html/body/div[6]/div[3]/div/div/div[2]/div/div/main/section[3]/div[3]/ul/li/div/div[2]')
                li_element = self.driver.find_element(By.XPATH, value=f'//*[@id="profile-content"]/div/div[2]/div/div/main/section[{section_number}]/div[3]/ul/li/div/div[2]')
                li_element_source = BeautifulSoup(li_element.get_attribute('outerHTML'),'html.parser')
                span_tags = li_element_source.find_all('span', class_='visually-hidden')
                span_text = [tag.text.strip() for tag in span_tags]
                info_list.append(span_text)
            else:
                for j in range(number_of_li):
                    li_number = j+1
                    # li_element = driver.find_element(By.XPATH, value=f'//*[@id="profile-content"]/div/div[2]/div/div/main/section[{section_number}]/div[3]/ul/li[{li_number}]/div/div[2]')
                    li_element = self.driver.find_element(By.XPATH, value=f'//*[@id="profile-content"]/div/div[2]/div/div/main/section[{section_number}]/div[3]/ul/li[{li_number}]/div/div[2]')
                    li_element_source = BeautifulSoup(li_element.get_attribute('outerHTML'),'html.parser')
                    span_tags = li_element_source.find_all('span', class_='visually-hidden')
                    span_text = [tag.text.strip() for tag in span_tags]
                    info_list.append(span_text)    

        return info_list
    #endregion


    #region GET EXPERIENCE
    def get_experience(self,sections: dict, full_xpath: str) -> list:
        if 'Experience' in sections:
            section_number = sections['Experience']
            section_info = self.get_section_info(section_number, full_xpath)
            return section_info

        return None
    #endregion


    #region GET SKILLS
    def get_skills(self, sections: dict, full_xpath: str) -> list:
        if 'Skills' in sections:
            section_number = sections['Skills']
            section_info = self.get_section_info(section_number, full_xpath)
            return section_info

        return None
    #endregion


    #region GET EDUCATION
    def get_education(self, sections: dict, full_xpath: str) -> list:
        if 'Education' in sections:
            section_number = sections['Education']
            section_info = self.get_section_info(section_number, full_xpath)
            return section_info

        return None
    #endregion


    #region GET CERTIFICATION
    def get_cert(self, sections: dict, full_xpath: str) -> list:
        if 'Licenses & certifications' in sections:
            section_number = sections['Licenses & certifications']
            section_info = self.get_section_info(section_number, full_xpath)
            return section_info

        return None
    #endregion


    #region GET PROFILE INFO
    def get_profile_info(self, profile_url) -> dict:
        self.driver.get(profile_url)
        main_element = self.driver.find_element(By.CLASS_NAME, value='scaffold-layout__main')
        full_xpath = get_fullxpath(self.driver,main_element)
        sections = self.get_sections()
        profile_info = {
            'Contact Info': self.get_contact_info(),
            'Experience': self.get_experience(sections, full_xpath),
            'Education': self.get_education(sections, full_xpath),
            'Skills': self.get_skills(sections, full_xpath),
            'Licenses & certifications': self.get_cert(sections, full_xpath)
        }

        return profile_info
    #endregion


    #region GET MUILIPLE PROFILES
    def get_profile_info_multi(self):
        connection_info_list = []
        i = 1
        for profile_url in self.connection_urls:
            profile_info = self.get_profile_info(profile_url)
            connection_info_list.append(profile_info)
            print(f"GET profile {i}")
            i += 1

        self.connection_info_cache = connection_info_list
        return connection_info_list
    #endregion


### TEST

In [55]:
# URL = 'https://www.linkedin.com/in/selenado/'
# URL = 'https://www.linkedin.com/in/trieuchinh/'
# URL = 'https://www.linkedin.com/in/tr%E1%BA%A7n-ho%C3%A0ng-qu%C3%A2n-37817ab6/'
# URL = 'https://www.linkedin.com/in/truong-nguyen-nhat-a5b77878/'
URL = 'https://www.linkedin.com/in/le-ha-b7a86997/'
password = "prodn123"
username = "skytdv123@gmail.com"


#### Test get section info method

In [47]:
crawler = LinkedInCrawler()
crawler.login_linkedin(username,password)
crawler.driver.get(URL)
sections = crawler.get_sections()
main_element = crawler.driver.find_element(By.CLASS_NAME, value='scaffold-layout__main')
full_xpath = get_fullxpath(crawler.driver, main_element)
section_info = crawler.get_cert(sections, full_xpath)

- Finish initializing a self.driver
- Finish importing the login credentials
- Finish keying in email
- Finish keying in password
- Finish Task: Login to Linkedin
Reached the bottom of the page.


#### Test get 1 profile info

In [62]:
profile_info = crawler.get_profile_info(URL)

In [63]:
profile_info

{'Contact Info': {'h1_tag_text': ['Le Ha'],
  'h3_tag_text': ['Le’s Profile',
   'Email',
   'Birthday',
   'Connected',
   'Get up to 4.6x replies when you message with InMail'],
  'a_tag_text': ['linkedin.com/in/le-ha-b7a86997',
   'lethiha.hr@gmail.com',
   'Try Premium for ₫0'],
  'span_tang_text': ['Dialog content start.',
   '',
   'May 20',
   'Jun 9, 2020',
   'Dialog content end.']},
 'Experience': [['Talent Acquisition Manager',
   'FPT Software Career · Full-time',
   'Jan 2020 to Present · 4 yrs 11 mos'],
  ['FPT Software',
   '12 yrs 9 mos',
   'Recruiter Specialist',
   'Full-time',
   'Mar 2013 to Present · 11 yrs 9 mos',
   'HA Noi',
   'Mar 2012 to Present · 12 yrs 9 mos']],
 'Education': [['Vietnamese commercal university',
   'Human Resources Management/Personnel Administration, General',
   '2009 - 2013',
   'Activities and societies: volunteer'],
  ['aa',
   "Master's Degree, Human Resources Management/Personnel Administration, General",
   '2009 - 2014',
   'light

#### Test get muiltiple profiles

In [ ]:
crawler = LinkedInCrawler()
crawler.login_linkedin(username,password)
crawler.get_connection_urls()
connection_info_list = crawler.get_profile_info_multi()

In [54]:
connection_info_list

NameError: name 'connection_info_list' is not defined

In [14]:
class Person:
    name: str

    def __init__(self):
        self.name = 'Quyen'
        

In [15]:
person = Person()
person.name

'Quyen'